In [5]:
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
import inspect

In [ ]:
#Layer norm
class LayerNorm(nn.Module):
    def __init__(self, ndim):
        super().__init__()
        self.weight= nn.Parameter(torch.ones(ndim)) #weight is a vector, not a matrix- elemnent vise scaling happens when Wx+b happens- if W were a matrix, it eats the purpose of norm and it would just become a linear layer.
        self.bias=nn.Parameter(torch.zeros(ndim))

    def forward(self,x):
        return F.layer_norm(x,self.weight.shape,self.weight, self.bias,1e-5)
    


#causal/masked self attention head
class Head(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.head_size=config.n_embed // config.n_head
        self.k=nn.Linear(config.n_embed, config.n_embed // config.n_head, bias=config.bias)
        self.q=nn.Linear(config.n_embed, config.n_embed // config.n_head, bias=config.bias)
        self.v=nn.Linear(config.n_embed, config.n_embed // config.n_head, bias=config.bias)
        self.register_buffer("tril",torch.tril(torch.ones(config.block_size,config.block_size)))
        self.dropout=nn.Dropout(config.dropout)

    def forward(self, x):
        B,T,C= x.size()
        #head_size= n_embed/n_head
        q=self.q(x)
        k=self.k(x)
        v=self.v(x) #BxTxhead_size
        wei= q @ k.transpose(-2,-1)* (1.0 / math.sqrt(k.size(-1))) #BxTxT
        wei=wei.masked_fill(self.tril[:T,:T]==0,float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei=self.dropout(wei)
        out= wei @ v  #BxTxT @ BxTxhead_size= BxTxhead_size
        return out

class MaskedMultiAtt(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.heads= nn.ModuleList([Head(config) for _ in range(config.n_head)])
        self.proj= nn.Linear(config.n_embed,config.n_embed)
        self.dropout=nn.Dropout(config.dropout)

    def forward(self, x):
        out= torch.cat([h(x) for h in self.heads], dim=-1) #concatenates the all the n_head heads of size n_embed/n_heads --> output is BxTxC
        out = self.dropout(self.proj(out))
        return out 



#Multi-Layer-Perceptron
class MLP(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.c_fc=nn.Linear(config.n_embed,4* config.n_embed)   #probably should change this 4x to 3x or smaller
        self.gelu= nn.GELU()
        self.proj= nn.Linear(4* config.n_head, config.n_embed)
        self.dropout=nn.Dropout(config.dropout)

    def forward(self,x):
        x=self.c_fc(x)
        x=self.gelu(x)
        x=self.proj(x)
        x=self.dropout(x)
        return x

#Block
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1=LayerNorm(config.n_embed,config.bias)
        self.attn=MaskedMultiAtt(config)
        self.ln2=LayerNorm(config.n_embed,config.bias)
        self.mlp= MLP(config)

    def forward(self,x):
        x=x+self.attn(self.ln1(x))
        x=x+self.mlp(self.ln2(x))
        return x

#defining size of model and structure
@dataclass
class SLMconfig:
    n_embed:int =384
    vocab_size:int =65
    block_size:int =64
    n_layer:int =4
    n_head:int =6
    bias: bool= False
    dropout: float= 0.0


#final architecture
class SLM(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config=config

        self.transformer= nn.ModuleDict(dict(
            wt_tok_em=nn.Embedding(config.vocab_size,config.n_embed),
            wt_pos_emb=nn.Embedding(config.block_size,config.n_embed),
            h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f=LayerNorm(config.n_head)

        ))
        self.ln_head=nn.Linear(config.n_embed,config.n_vocab,bias=False)
